In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def convert_to_datetime(date_string):
    norsk_to_engelsk = {
        'mandag': 'Monday', 'tirsdag': 'Tuesday', 'onsdag': 'Wednesday',
        'torsdag': 'Thursday', 'fredag': 'Friday', 'lørdag': 'Saturday',
        'søndag': 'Sunday', 'januar': 'January', 'februar': 'February',
        'mars': 'March', 'april': 'April', 'mai': 'May', 'juni': 'June',
        'juli': 'July', 'august': 'August', 'september': 'September',
        'oktober': 'October', 'november': 'November', 'desember': 'December'
    }
    for norsk, engelsk in norsk_to_engelsk.items():
        date_string = date_string.replace(norsk, engelsk)
    return datetime.strptime(date_string + ' 2024', '%A %d. %B %Y')

def scrape_numbers():
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    hovedtall = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')][:5]
    stjernetall = [int(li.text.strip()) for li in soup.select('ul li.h-10.w-10.rounded-full')][5:]
    
    date = soup.select_one('span.font-medium.font-jubel').text.strip()
    
    return date, hovedtall, stjernetall

# Sett opp Chrome-opsjoner for headless-modus
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--window-size=1920x1080")

# Sett opp Selenium WebDriver med Chrome-opsjoner
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.norsk-tipping.no/lotteri/eurojackpot/resultater")

results = []

while True:
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul li.h-10.w-10.rounded-full'))
        )
        
        date, hovedtall, stjernetall = scrape_numbers()
        results.append({'Dato': date, 'Hovedtall': hovedtall, 'Stjernetall': stjernetall})
        print(f"Scraped: {date}")
        
        previous_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "results-selected-draw-previous"))
        )
        previous_button.click()
        
    except TimeoutException:
        print("Kan ikke klikke mer eller siden lastet ikke. Avslutter.")
        break

driver.quit()

# Opprett DataFrame
results_df = pd.DataFrame(results)

# Konverter 'Dato' til datetime
results_df['Dato'] = results_df['Dato'].apply(convert_to_datetime)

# Sorter DataFrame etter dato i synkende rekkefølge
sorted_df = results_df.sort_values(by='Dato', ascending=True)

# Vis sortert DataFrame
print(sorted_df)

Scraped: tirsdag 12. november
Scraped: fredag 8. november
Scraped: tirsdag 5. november
Scraped: fredag 1. november
Scraped: tirsdag 29. oktober
Scraped: fredag 25. oktober
Scraped: tirsdag 22. oktober
Scraped: fredag 18. oktober
Scraped: tirsdag 15. oktober
Scraped: fredag 11. oktober
Scraped: tirsdag 8. oktober
Scraped: fredag 4. oktober
Scraped: tirsdag 1. oktober
Scraped: fredag 27. september
Scraped: tirsdag 24. september
Scraped: fredag 20. september
Scraped: tirsdag 17. september
Scraped: fredag 13. september
Scraped: tirsdag 10. september
Scraped: fredag 6. september
Scraped: tirsdag 3. september
Scraped: fredag 30. august
Scraped: tirsdag 27. august
Scraped: fredag 23. august
Scraped: tirsdag 20. august
Scraped: fredag 16. august
Scraped: tirsdag 13. august
Scraped: fredag 9. august
Scraped: tirsdag 6. august
Scraped: fredag 2. august
Kan ikke klikke mer eller siden lastet ikke. Avslutter.
         Dato             Hovedtall Stjernetall
29 2024-08-02  [14, 20, 26, 30, 31]      

In [11]:
# Assuming sorted_df is your DataFrame

# Define the set of draws
draws_set = list(range(len(sorted_df)))
draws_output = f"# Define the set of draws\nset DRAWS := {' '.join(map(str, draws_set))};"

# Generate Hovedtall output
hovedtall_output = "# Hovedtall data (5 numbers per draw)\nparam Hovedtall : 1 2 3 4 5 :="
for index, row in sorted_df.iterrows():
    hovedtall_numbers = ' '.join(f"{num:4d}" for num in row['Hovedtall'])
    hovedtall_output += f"\n   {index:2d} {hovedtall_numbers}"

# Generate Stjernetall output
stjernetall_output = "# Stjernetall data (2 numbers per draw)\nparam Stjernetall : 1 2 :="
for index, row in sorted_df.iterrows():
    stjernetall_numbers = ' '.join(f"{num:4d}" for num in row['Stjernetall'])
    stjernetall_output += f"\n   {index:2d} {stjernetall_numbers}"

# Combine all outputs
final_output = f"{draws_output}\n\n{hovedtall_output}\n;\n\n{stjernetall_output}\n;"

# Print or save the final output
print(final_output)

# Optionally, save to a file
# with open('lottery_data.dat', 'w') as f:
#     f.write(final_output)


# Define the set of draws
set DRAWS := 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29;

# Hovedtall data (5 numbers per draw)
param Hovedtall : 1 2 3 4 5 :=
   29   14   20   26   30   31
   28    4    9   22   32   35
   27   15   18   25   29   35
   26    6    9   33   34   50
   25    4   11   16   46   50
   24   25   28   29   31   33
   23   13   21   22   26   48
   22   15   24   29   33   39
   21    8   11   25   31   48
   20    5   17   23   36   37
   19    7   11   27   42   45
   18    7   10   31   41   46
   17    2    3   17   40   44
   16    1    3   13   24   44
   15    9   17   19   26   39
   14    3   13   34   41   43
   13    6   15   25   29   41
   12   17   37   42   45   50
   11    4   16   27   34   44
   10   13   29   42   44   48
    9    8   11   23   44   45
    8    9   20   38   44   45
    7    2    4   23   30   40
    6    4   32   36   38   47
    5    2   19   36   42   50
    4    3   17   26   30   49
    